# Generative perspective on State Space Models with spike train observations

In this notebook, we will play around with parameters of a state space model and generate various spike trains.
These population activity patterns will be fun to look at. (hopefully)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# set a default figure size for matplotlib figures (using 1 + golden ratio)
plt.rcParams['figure.figsize'] = (10 * np.array([1, 1 / (1 + (np.sqrt(5)+ 1)/2)])).tolist()
# set a default font size for matplotlib figures
plt.rcParams['font.size'] = 12

## Neural Manifolds and Causality

Neurons cause the coordinated activity pattern that we experimentally observe. However, with the current experimental technology, we do not yet have enough data to recover the full spiking neural network faithfully.

Fortunately, neural recordings have a lot of spatial structure that restricts activity to a low-dimensional manifold.
Moreover, neural recordings have a lot of temporal structure to define an effective "flow" on the manifold.

Therefore, to analyze the neural data, we could focus on the effective collective behavior of the population reflected in neural recordings.
We assume we have access to the low-dimensional population state via the partially observed neurons.
Think of the neurons as noisy measurement devices that are coupled with the thinig we really want to measure, the neural population state.

This means, we can define an observation model: $p(y(t) \mid x(t))$ where $y(t)$ is the neural data and $x(t)$ is the latent (i.e. hidden or unobserved) neural state. $x(t)$ lives on the coordinate system that parametrizes the neural manifold or its embedding space.

This may may look acausal, but it makes sense in the statistical sense. Using this model does not assume that the neurons are not fundamentally generating the latent state dynamics, it's merely a methodological necessity.

For educational purposes, it is useful to generate spike trains as if this acausal model is true.
It allows to gain intuition about what the statistical model expects the observed data to be if the assumptions hold up.

## A simple 1-D latent (neural) trajectory

For illustration, we will use a sinusoid as the 1-D latent trajectory.
$$ x(t) = sin(2\pi f\cdot t) $$
In this example, $x(t)$ represents the (instantaneous) state of the neural population of interest.

In [ ]:
# simulate a simple latent trajectory
nT = 1000  # number of time points (samples) in the simulated trajectory
T = 10 # duration of the trajectory in seconds
frq = 0.3 # frequency of the sinusoid in Hz
tr = np.linspace(0, T, nT) # time range
dt = tr[1] - tr[0] # time step in seconds
x = np.sin(2 * np.pi * frq * tr) # generate a sinusoid over time
fig = plt.figure(); plt.plot(tr, x); plt.title('1-D latent trajectory'); plt.xlabel('time (s)');

## One Poisson neuron driven by the latent process

We will generate spike trains from a *Poisson neuron*, which is just an inhomogeneous Poisson process with a time varying firing rate function $\lambda(t)$.
The spike count $y(t)$ in a small time bin of size $\Delta$ is distributed as the Poisson distribution:
$$ y(t) \sim \text{Poisson}(\Delta\lambda(t)) $$

Importantly, the firing rate will be a function of $x(t)$, but not of past $x$ nor past $y$.
$$ \lambda(t) = g(x(t)) $$
The only constraint for $g(\cdot)$ is that the resulting firing rate has to be non-negative.
A mathematically convenient function is the exponential function and relates to the spike-response model (Gerstner & Kistler 2002).

$$ \lambda(t) = \exp(a x(t) + b) = \exp(b)\exp(a x(t)) $$

- Gerstner, W., & Kistler, W. M. (2002). Spiking Neuron Models. Cambridge University Press. https://doi.org/10.1017/cbo9780511815706

In [ ]:
# plot the log-linear Poisson neuron's tuning curve
# y-axis is the firing rate, x-axis is the signal to be encoded
a = 5
b = -3
lam = np.exp(a * x + b)

plt.figure(figsize=(6, 4))
plt.plot(x, lam)
plt.xlabel('x (latent process)')
plt.ylabel('Firing rate λ(x)')
plt.title('Log-linear Poisson neuron tuning curve')
plt.grid(True)
plt.show()

In [ ]:
y = np.random.poisson(lam*dt)

plt.figure(figsize=(10, 2))
plt.plot(tr, lam, label='firing rate');
plt.eventplot(np.nonzero(y)[0]/nT*T, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.legend();

## A population of Poisson neurons driven by a common 1-D latent process

We can have more than one neuron that's driven by the same latent process.
This way, we an have more observation dimensions than the latent state space dimension.
Let's given them a random amount of "drive".

In [ ]:
nNeuron = 200
C = 2 * np.random.randn(nNeuron)
b = -2.0 + np.random.rand(nNeuron,1)
lam = np.exp(np.outer(C, x) + b)
y = np.random.poisson(lam*dt)

We can make spike raster plot. But since we know the amount of drive (each value in $C$), we can sort the neurons accordingly as well.

In [ ]:
cidx = np.argsort(C)

raster = []
rasterSorted = []
for k in range(nNeuron):
    raster.append(np.nonzero(y[k,:])[0]/nT*T)
    rasterSorted.append(np.nonzero(y[cidx[k],:])[0]/nT*T)

plt.subplots(1,2, figsize=(10, 4))
plt.subplot(1,2,1)
plt.eventplot(raster, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot'); plt.ylabel('neurons');
plt.subplot(1,2,2)
plt.eventplot(rasterSorted, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot (again)'); plt.ylabel('sorted neurons');

## 2D latent space example

Here we will build a 2D manifold with two independent processes.
The first latent dimension will be same as above, but we will add $x_2(t)$ as a sawtooth function:
$$ x_2(t) = t \,\, \text{mod} \, 1 $$

In [ ]:
x2 = 1.5 * ((tr % 1) - 0.5)
X = np.vstack([x, x2]) # (latent dim) x (time)

In [ ]:
plt.subplots(2,1,figsize=(10,4))
plt.subplot(2,1,1);
plt.plot(tr, x ); plt.ylabel('first latent dim'); plt.xlabel('time')
plt.subplot(2,1,2);
plt.plot(tr, x2); plt.ylabel('second latent dim'); plt.xlabel('time')

Now that we have more than 1 latent dimension, we face a choice of how neurons relate to each of the latent dimes.

In [ ]:
X.shape

In [ ]:
#!git clone https://github.com/catniplab/neurofisherSNR.git
#!pip install ./neurofisherSNR

from neurofisherSNR.optimize import optimize_C

nNeuron = 100
dLatent = X.shape[0]
snr_db = -1

C = 2 * np.random.randn(dLatent, nNeuron) # random projection
b = np.random.randn(nNeuron) - np.log(0.01)

C, b, snr = optimize_C(X.T, C, b, tgt_rate_per_bin=0.01, tgt_snr=snr_db, max_rate_per_bin=5)

### Random projection observation

Random projection assumes that each neuron is deriven by all the latent dimensions with a random amount.
Under this assumption, the neural manifold is likely oblique to the axes, i.e., the neuron will be modulated by changes in any direction in the latent state space.
Theoretical analysis of [Gao & Ganguli 2015] assumes random projections and showed that not many neurons need to be sampled (observed) to recover the manifold structure.
In addition, the so-called *mixed-selectivity* [Tye et al. 2024] appears as a result.

- Gao, P., & Ganguli, S. (2015). On Simplicity and Complexity in the Brave New World of Large-Scale Neuroscience. Current Opinion in Neurobiology, 32, 148–155.
- Tye, K. M., Miller, E. K., Taschbach, F. H., Benna, M. K., Rigotti, M., & Fusi, S. (2024). Mixed selectivity: Cellular computations for complexity. Neuron, 112(14), 2289–2303. https://doi.org/10.1016/j.neuron.2024.04.017

In [ ]:
#dLatent = X.shape[0]
#C = 2 * np.random.randn(nNeuron, dLatent) # random projection
lam = np.exp(X.T @ C + b)
y = np.random.poisson(lam*dt)

In [ ]:
cidx1 = np.lexsort((C[:,0], C[:,1]), axis=0)
cidx2 = np.lexsort((C[:,1], C[:,0]), axis=0)

In [ ]:
raster = []; rasterSorted1 = []; rasterSorted2 = []
for k in range(nNeuron):
    raster.append(np.nonzero(y[k,:])[0]/nT*T)
    rasterSorted1.append(np.nonzero(y[cidx1[k],:])[0]/nT*T)
    rasterSorted2.append(np.nonzero(y[cidx2[k],:])[0]/nT*T)

plt.subplots(1,3, figsize=(10, 3))
plt.subplot(1,3,1)
plt.eventplot(raster, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot'); plt.ylabel('neurons');
plt.subplot(1,3,2)
plt.eventplot(rasterSorted1, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot (1)'); plt.ylabel('sorted neurons');
plt.subplot(1,3,3)
plt.eventplot(rasterSorted2, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot (2)'); plt.ylabel('sorted neurons');

### Axis aligned observation

Biologists have long loved neurons that are tuned specifically for a particular feature but not modulated by others.
In our context, the neurons will be either driven by the first dimension or the second dimension of the latent process.
Recent paper argues that this is optimal [Whittington et al. 2022].

 - Whittington, J. C. R., Dorrell, W., Ganguli, S., & Behrens, T. E. J. (2022). Disentangling with Biological Constraints: A Theory of Functional Cell Types. In arXiv [q-bio.NC]. arXiv. http://arxiv.org/abs/2210.01768

In [ ]:
C = 2.0 * np.random.randn(nNeuron, dLatent)
b = -2.0 + np.random.rand(nNeuron,1)
bidx = np.random.rand(nNeuron) < 0.5
C[bidx, 0] = 0
C[~bidx, 1] = 0
b[bidx] += 1.5 # boost the firing rate a bit for the 2nd latent dim
lam = np.exp(C @ X + b)
y = np.random.poisson(lam*dt)

In [ ]:
cidx = np.lexsort((C[:,1], C[:,0]), axis=0)

In [ ]:
raster = []
rasterSorted = []
for k in range(nNeuron):
    raster.append(np.nonzero(y[k,:])[0]/nT*T)
    rasterSorted.append(np.nonzero(y[cidx[k],:])[0]/nT*T)

plt.subplots(1,2, figsize=(10, 4))
plt.subplot(1,2,1)
plt.eventplot(raster, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot'); plt.ylabel('neurons');
plt.subplot(1,2,2)
plt.eventplot(rasterSorted, lw=0.5, color='k', label='spikes')
plt.xlim(0, T); plt.xlabel('time'); plt.yticks([]); plt.title('raster plot (again)'); plt.ylabel('sorted neurons');

# Signal-to-Noise Ratio of population spike trains

https://github.com/catniplab/neuro-fisher


## Some dynamical law governing the latent states

So far, the latent states were given and not generated in a Markovian manner, that is, given the current state $x(t)$, the future states were not a function of $x(t-1)$ (or further past).
In general, a dynamical law can be represented as a *dynamical system* or an ordinary differential equation (ODE):
$$ \frac{dx}{dt} = \dot{x} = f(x(t)) $$
where $f()$ is a smooth function that represents the vector field.

### van der Pol oscillator
Van der pol oscillator is defined as 2D dimensional first order differential equations:
    $$ \dot{x} = y$$
    $$ \dot{y} = \mu(1-x^2)y -x $$

For our simulations we take a discrete time grid for convenience.
We use an Euler integration of a Van der Pol oscillator with noisy transitions with $\mu=1.5$, $\tau_1=0.1$, $\tau_2=0.1$, and $\sigma=0.1$:

$$ x_{t+1,1} = x_{t,1} + \tau_1^{-1} \Delta x_{t,2} + \sigma \epsilon$$
$$ x_{t+1,2} = x_{t,2} + \tau_2^{-1} \Delta(\mu (1-x_{t,1})^2 x_{t,2} - x_{t,1}) + \sigma \epsilon$$

For the sake of brevity in the notebook, the generation code is provided in `code_pack/genereate_vdp_data.py` and we only load the saved data here.

In [ ]:
import h5py
import scipy
import scipy.ndimage

from code_pack.plotting import plot_two_d_vector_field_from_data, raster_to_events
from code_pack.generate_vdp_data import generate_van_der_pol, generate_noisy_van_der_pol

# loading data from ./data/vdp_noisy.h5
file_name = "vanderpol/data/poisson_obs.h5"

# dynamics parameters
data = h5py.File(file_name, 'r')
system_parameters = {}
system_parameters['mu'] = data['mu']
system_parameters['tau_1'] = data['tau_1']
system_parameters['tau_2'] = data['tau_2']
system_parameters['sigma'] = data['sigma']
system_parameters['scale'] = np.array(data['scale'])

Y = np.array(data['Y'])
X = np.array(data['X'])
C = np.array(data['C'])
b = np.array(data['bias'])

n_trials = Y.shape[0]
n_latents = X.shape[2]
n_neurons = Y.shape[2]
n_time_bins = Y.shape[1]

### Visualizationing trajectories

In [ ]:
# plotting trajectories of the dataset
fig, ax = plt.subplots(1, 1, figsize=(5,5))
_ = ax.plot(X[0,:,0], X[0,:,1])
ax.scatter(X[0, 0, 0], X[0, 0, 1], marker='o', color='red', zorder=10, s=100, label='start')
ax.scatter(X[0, -1, 0], X[0, -1, 1], marker='x', color='red', zorder=10, s=100, label='end')

# system_parameters_copy = copy.deepcopy(system_parameters)
system_parameters['sigma'] = 0.0
dynamic_func = lambda inp : generate_noisy_van_der_pol(inp, np.array([0.0, 5e-3]), system_parameters)
axs_range = {'x_min':-1.5, 'x_max':1.5, 'y_min':-1.5, 'y_max':1.5}
plot_two_d_vector_field_from_data(dynamic_func, ax, axs_range)

ax.legend()
ax.set_title('sample trajectory (true state)');

### Effect of tuning function

TODO: explain different inverse link functions (soft plus)

In [ ]:
C_tilde = np.array(data['C_tilde'])
idx = np.lexsort((C_tilde[:, 0], C_tilde[:, 1]), axis=0)  # sort the loading

# showing the spike raster generated from noisy Vdp
fig, axs = plt.subplots(1, 3, figsize=(15, 3), sharex=True, sharey=True)
events = raster_to_events(np.array(data['Y'])[0, :, :])
events_softplus = raster_to_events(np.array(data['Y_softplus'])[0, :, :])
events_axis_aligned = raster_to_events(np.array(data['Y_axis'])[0, :, idx].transpose())
axs[0].eventplot(events, linewidths=0.5, color='k');
axs[1].eventplot(events_softplus, linewidths=0.5, color='k');
axs[2].eventplot(events_axis_aligned, linewidths=0.5, color='k');
axs[0].set_title(f'exp()$');
axs[1].set_title(f'softplus$()$');
axs[2].set_title(f'axis aligned');
axs[0].set_xlabel("Time");
axs[0].set_ylabel("Neuron");

## What's next?

Now we understand better the generative process of the model. But what we are interested is the opposite direction, that is, how do we infer the model parameters given just the observations (neural data)? This is the statistical inference problem of interest.